In [38]:
from transformers import PreTrainedTokenizerFast
import pandas as pd
import re

In [23]:
data = pd.read_csv("dataset/crawling_poem_100000.csv")

In [ ]:
# 크롤링한 데이터에 대해 아래의 처리 과정을 수행한다.
# 아래의 사항들은 크롤링한 데이터 중 5000개를 검수해보고 찾아낸 패턴들이다.
# 1. "poem"에서 NaN이 존재하니 삭제
# 2. "poem"에 작가 이름이 존재하는 시(약 800개) 제거
# 3. "poem" 끝부분에 숫자id, 날짜가 존재하는 패턴 제거
# 4. 아래 시인들의 시는 학습에 오히려 방해를 주기에 제거
#   - "윤봉택" 시인의 시 모두 제거 (제주도 방언을 너무 사용.)
#   - "정웅" 시인의 시 모두 제거 (한자를 너무 사용하고, 단어에 대한 내용정리가 존재.)
#   - "천숙녀" 시인의 시 모두 제거 (한자를 너무 사용.)
# 5. 아래의 시들은 학습에 방해를 주기에 제거.
#   - 2022년 : 이영지
#   - 바다와 벚꽃이 있는 연극무대 : 박종인
#   - 시리즈
# 7. 아래의 출처 패턴들을 제거
#   - "서울시정일보 ...$"
#   - "문학바탕$"
#   - "스토리문학관$"
#   - "문학과 사람들$"
# 8. 다양한 작가 이름 패턴을 제거 ex) 윤 일환, 윤 일 환
# 9. "이 게시물은...$"패턴 제거
# 10. 시의 각 행들의 length가 일정 범위를 벗어나는 경우, 삭제
#   - 범위는 Metaphor_Gen의 각 데이터셋의 길이 분포이며, 분포는 아래와 같다.
#       - min : 10
#       - 25% : 18
#       - 50% : 19
#       - 75% : 21
#       - max : 32

In [24]:
data

,title,author,poem,num_stanza,num_line,first_line
0,모래와 바위,조남명,베푼 일은\n바닷가 모래에 써\n밀물에 지우고\n\n받은 은혜는\n바위에 새겨\n잊...,4,2,베푼 일은
1,왜 사랑은 아픈가,홍수희,내 마음 통째 내어주기 어디 그리 쉬운가\n내 목숨 온새미 내어주기 어디 그리 쉬운...,1,7,내 마음 통째 내어주기 어디 그리 쉬운가
2,누군가에게 먼저 마음을 빌려준 초록색의 어려움,정세일,누군가에게 먼저 마음을 빌려준\n초록색의 어려움\n그럼에도 한마디 말을 하지 않고 ...,1,4,누군가에게 먼저 마음을 빌려준
3,능소화 사랑,김용화,단연코 잊지 않으리라\n달빛 따서 덮어주던 \n그날 밤 그대 다순 손길\n\n잠든 ...,4,3,단연코 잊지 않으리라
4,능소화,김용화,가까이 오지 마셔요\n이윽한 눈빛으로 \n바라보지 마셔요\n\n애오라지 단 한 분\...,3,3,가까이 오지 마셔요
...,...,...,...,...,...,...
88555,돼지의 넉살,하재청,하 재 청\n지구의 반대편에서 \n시계추가 흔들리고 \n수요일 오후가 잠자고 있다 ...,4,3,하 재 청
88556,원근법,하재청,하 재 청\n\n속살을 뚫고 나와 \n멀리 달아나는 풍경들 \n강 건너 산 아래로 ...,3,3,하 재 청
88557,얼굴 없는 원탁,하재청,하 재 청\n금테안경이 반짝인다 \n빈 의자가 점잖게 활극을 펼친다 \n네 칼라는 ...,1,3,하 재 청
88558,배꼽티,하재청,하 재 청 \n빨랫줄에 \n배꼽티가 걸려 있다 \n배꼽을 불고 있는 아이들 \n배꼽...,1,4,하 재 청


In [25]:
# NaN 제거 (1개 제거)
data = data.dropna(subset=["poem"])
data = data.reset_index(drop=True) 

In [26]:
# "poem"에 "author"가 존재하는 시들은 제거. 88560 -> 87700 (859개 제거)
data = data[data.apply(lambda row: str(row['author']) not in str(row['poem']), axis=1)]
data = data.reset_index(drop=True) 

In [27]:
# "poem"끝에 숫자로 끝나는 패턴 제거.
# 2024 01 31 ~~~(날짜) / 2024년 / 323129323012 ~~~(고유id)
for i in range(len(data)):
    match1 = re.search(r"\d{4} \d{1,2} \d{1,2}", data["poem"][i])
    if match1:
        data.iloc[i, 2] = data["poem"][i][:match1.start()].strip()
    match2 = re.search(r"\d{5,}", data["poem"][i])
    if match2:
        data.iloc[i, 2] = data["poem"][i][:match2.start()].strip()
    
    data.iloc[i, 2] = re.sub(r"\d{4}년$", "", data["poem"][i]).strip()

    while True: # poem 끝에 숫자로 마무리되지 않을 때까지 계속해서 삭제
        match3 = re.search(r"\d{1,}$", data["poem"][i])
        if match3:
            data.iloc[i, 2] = re.sub(r"\d{1,}$", "", data["poem"][i]).strip()
        else:
            break

In [28]:
# "윤봉택", "정웅", "천숙녀" 시인의 시 데이터 제거
data = data[~(data["author"].isin(["윤봉택", "정웅", "천숙녀"]))]
data = data.reset_index(drop=True) 

In [29]:
# "시리즈"라는 단어가 포함된 제목들의 시는 글 안에 여러 개의 시들이 존재해 구분해주기 어려워 삭제
data = data[~data["title"].str.contains("시리즈")]
data = data.reset_index(drop=True)

# 학습에 방해되는 특정 시들 제거
data = data[~(data["title"].isin(["2022년", "바다와 벚꽃이 있는 연극무대"]))]
data = data.reset_index(drop=True)

In [30]:
# 출처 패턴들과 시의 제목을 제거
# "서울시정일보..." : 359개
# "문학바탕" : 115개
# "스토리문학관" : 127개
# "문학과 사람들" : 79개
# "계간 문학..." : 
for i in range(len(data)):
    match1 = re.search(r"서울시정일보|문학바탕|스토리문학관|스토리문학|문학과 사람들|계간 문학", data["poem"][i])
    if match1:
        data.iloc[i, 2] = data["poem"][i][:match1.start()].strip()
    match_title = re.search(r" - 문학바탕| - 스토리문학관| - 문학과 사람들", data["title"][i])
    if match_title:
        poem_title = re.sub(r"^[가-힣0-9\s\n]", "", data["title"][i][:match_title.start()])
        data.iloc[i, 2] = re.sub(f"^{poem_title}", "", data["poem"][i]).strip()


In [31]:
# 다양한 작가 이름 패턴과 "이 게시물은..."패턴 제거
for i in range(len(data)):
    match = re.search(r"이 게시물은", data["poem"][i])
    if match:
        data.iloc[i, 2] = data["poem"][i][:match.start()].strip()
    # author의 길이가 3인 경우에만 일단 제거
    if (isinstance(data["author"][i], str)) and (len(data["author"][i]) == 3):
        data.iloc[i, 2] = re.sub(" ".join(data["author"][i]), "", data["poem"][i]).strip()

In [33]:
# num_stanza, num_line, first_line을 수정
for i in range(len(data)):
    data.iloc[i, 3] = data["poem"][i].count("\n\n") + 1
    text_until_newline = re.split('\n', data["poem"][i], 1)[0]
    word_count = len(re.findall('\s', text_until_newline)) + 1
    data.iloc[i, 4] = word_count
    data.iloc[i, 5] = text_until_newline

In [34]:
# 각 행의 길이를 파악
min_num = 10
max_num = 32

df = {"title": [],
      "author": [],
      "poem": [],
      "num_stanza": []}

for i in range(len(data)):
    poem_data = str(data["poem"][i]).replace("\n\n", "\n").split("\n")          # 행 별로 분리해서 저장
    all_lines_valid = True
    for line in poem_data:
        if (len(line) < min_num) or (len(line) > max_num):    # 각 행의 길이가 범위 내에 있는 지 확인
            all_lines_valid = False
            break
    
    # 모든 행들이 범위 내에 있으면 df에 저장
    if all_lines_valid:
        df["title"].append(data["title"][i])
        df["author"].append(data["author"][i])
        df["poem"].append(data["poem"][i])
        df["num_stanza"].append(data["num_stanza"][i])

df = pd.DataFrame(df)

In [36]:
df

,title,author,poem,num_stanza
0,왜 사랑은 아픈가,홍수희,내 마음 통째 내어주기 어디 그리 쉬운가\n내 목숨 온새미 내어주기 어디 그리 쉬운...,1
1,봄 숨결,송정숙,평범한 하루의 시작\n커피향으로 공간을 채우니\n은은한 향이 몸 안으로 스민다\n\...,4
2,햇살 빛나는 인생,김덕성,벌거벗은 겨울나무에\n봄비 촉촉하고 포근하게 적시니\n환희의 흡족한 파란 미소\n\...,4
3,춘삼월 길에 서서,김덕성,창문을 노크하는 빗소리\n부드러운 봄 마중 소리 들려오고\n봄소식을 전하는 반가운 ...,3
4,정월 대보름,靑山 손병흥,허공 끝 나부끼던 숨가쁜 세월\n달집 태운 채 소원을 비는 모습\n\n나물 반찬과 ...,5
...,...,...,...,...
12789,노을 - 나태주,나태주,보아주는 이 없어서\n더욱 아리따운 아낙이여,1
12790,진눈깨비,하재청,더 이상의 결별은 없다 \n겨울비가 내린다 진눈깨비가 날린다 \n무엇도 아닌 것들 ...,1
12791,황강,하재청,가죽을 쫘악 벗기고 줄기를 꺾어 \n한 입 베어 물면 핏물이 피지직 \n우린 이런 ...,4
12792,불가사리,하재청,부두에 등대가 움직이자 \n어둠이 하얀 등뼈를 드러내고 일렁인다 \n고생대 푸른 어...,1


In [37]:
df.to_csv("poem_dataset.csv", index=False)